# Neural Networks

https://github.com/YonatanRA/Intro-redes-neuronales


![node](images/node.png)


![acti](images/acti.png)


![neural](images/neural.jpg)

### MLP (Multi-Layer Perceptron)

In [1]:
import warnings
warnings.simplefilter('ignore')

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score


from sklearn.neural_network import MLPClassifier

In [2]:
muon=pd.read_csv('../data/muon.csv')

muon.head()

,id,LifeTime,dira,FlightDistance,FlightDistanceError,IP,IPSig,VertexChi2,pt,DOCAone,...,p1_p,p2_p,p0_eta,p1_eta,p2_eta,SPDhits,production,signal,mass,min_ANNmuon
0,18453471,0.001578,0.999999,14.033335,0.681401,0.016039,0.451886,1.900433,1482.037476,0.066667,...,12290.760742,39264.398438,3.076006,4.003800,4.031514,458,-99,0,1866.300049,0.277559
1,5364094,0.000988,0.999705,5.536157,0.302341,0.142163,9.564503,0.865666,3050.720703,0.024022,...,16562.667969,7341.257812,3.228553,2.786543,2.975564,406,-99,0,1727.095947,0.225924
2,11130990,0.000877,0.999984,6.117302,0.276463,0.034746,1.970751,10.975849,3895.908691,0.055044,...,22695.388672,10225.309570,3.536903,2.865686,3.052810,196,-99,0,1898.588013,0.368630
3,15173787,0.000854,0.999903,5.228067,0.220739,0.076389,4.271331,3.276358,4010.781738,0.053779,...,16909.515625,9141.426758,3.087461,3.218034,2.375592,137,-99,0,1840.410034,0.246045
4,1102544,0.001129,0.999995,39.069534,1.898197,0.120936,4.984982,0.468348,4144.546875,0.004491,...,97612.804688,47118.785156,4.632295,4.711155,4.296878,477,-99,0,1899.793945,0.222060


In [3]:
X=muon.drop(columns=['id', 'production', 'signal', 'mass', 'min_ANNmuon'])

y=muon.signal

In [4]:
X_s=StandardScaler().fit_transform(X)

In [5]:
X_train, X_test, y_train, y_test=tts(X_s, y)

In [ ]:
mlp=MLPClassifier(hidden_layer_sizes=(100,), activation='tanh', max_iter=500)

mlp.fit(X_train, y_train)

In [ ]:
y_pred=mlp.predict(X_test)

y_prob=mlp.predict_proba(X_test)

In [ ]:
y_pred

In [ ]:
y_prob

In [ ]:
confusion=confusion_matrix(y_test, y_pred)
ax=sns.heatmap((confusion/len(y_pred)*100), annot=True)

b, t=ax.get_ylim()
ax.set_ylim(b+.5, t-.5)

plt.xlabel('prediccion')
plt.ylabel('verdad')
plt.title('confusion')
plt.show();

In [ ]:
fpr, tpr, umb=roc_curve(y_test, y_pred)
auc=roc_auc_score(y_test, y_pred)

plt.plot(fpr, tpr)
plt.plot(fpr, fpr, 'r--')
plt.title('ROC curve: AUC={}'.format(auc))
plt.show();

# esto esta mal, tiene que ser con y_prob

In [ ]:
len(y_prob)==len(y_test)

### CNN (Convolutional Neural Network)


https://en.wikipedia.org/wiki/Convolution


![neural2](images/neural2.webp)

![neural3](images/neural3.gif)

![neural4](images/neural4.png)

### CNN-1D

In [ ]:
import numpy as np

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D, MaxPooling1D

In [ ]:
def secuencia(lst, steps):
    
    X,y=[],[]
    
    for i in range(len(lst)):
        
        idx=i+steps
        
        if idx>len(lst)-1:break
            
        s_x, s_y=lst[i:idx], lst[idx]
        X.append(s_x)
        y.append(s_y)
        
    return np.array(X), np.array(y)

In [ ]:
raw_seq=[10, 20, 30, 40, 50, 60, 70, 80, 90]

steps=3

In [ ]:
X,y=secuencia(raw_seq, steps)

n_features=1

In [ ]:
X=X.reshape((X.shape[0], X.shape[1], n_features))

In [ ]:
# modelo

modelo=Sequential()
modelo.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(steps, n_features)))
modelo.add(MaxPooling1D(pool_size=2))
modelo.add(Flatten())
modelo.add(Dense(1))


modelo.compile(optimizer='adam', loss='mse')


modelo.fit(X, y, epochs=1000, verbose=0)

In [ ]:
modelo.summary()

In [ ]:
# prediccionº

x_input=np.array([50,60,70]).reshape((1, steps, n_features))

In [ ]:
y_pred=modelo.predict(x_input, verbose=0)

y_pred

### CNN-2D

In [ ]:
fashion=pd.read_csv('../data/fashion-mnist.csv')

fashion.head()

In [ ]:
plt.figure(figsize=(5,5))
plt.imshow(fashion.values[4,1:].reshape(28,28))
plt.show();

In [ ]:
fashion.label.unique()

In [ ]:
import keras

from keras.utils import to_categorical
from keras.layers import Dropout
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization

In [ ]:
input_shape=(28, 28, 1)

In [ ]:
X=np.array(fashion.iloc[:, 1:])

y=to_categorical(np.array(fashion.iloc[:,0]))

In [ ]:
X_train, X_test, y_train, y_test=tts(X, y)

In [ ]:
X_train=X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')/255

X_test=X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32')/255

In [ ]:
batch=256
n_clases=10
epocas=50

In [ ]:
# modelo

modelo=Sequential()
modelo.add(Conv2D(32, kernel_size=(3, 3), activation='relu',
                  kernel_initializer='he_normal', input_shape=input_shape))

modelo.add(MaxPooling2D((2,2)))
modelo.add(Dropout(0.25))

modelo.add(Conv2D(64, (3, 3), activation='relu'))
modelo.add(MaxPooling2D((2,2)))
modelo.add(Dropout(0.25))

modelo.add(Conv2D(128, (3, 3), activation='relu'))
modelo.add(Dropout(0.4))

modelo.add(Flatten())
modelo.add(Dense(128,  activation='relu'))
modelo.add(Dropout(0.3))
modelo.add(Dense(n_clases, activation='softmax'))


modelo.compile(loss=keras.losses.categorical_crossentropy, 
               optimizer=keras.optimizers.Adam(), 
               metrics=['accuracy'])

modelo.summary()

In [ ]:
historico=modelo.fit(X_train, y_train, batch_size=batch, epochs=epocas, verbose=0)

In [ ]:
score=modelo.evaluate(X_test, y_test, verbose=0)

In [ ]:
print ('Perdida testeo: ', score[0])
print ('Accuracy testeo: ', score[1])

In [ ]:
acc=historico.history['accuracy']
loss=historico.history['loss']

epocas=range(len(acc))

In [ ]:
plt.plot(epocas, acc, 'bo', label='train acc')
plt.plot(epocas, loss, 'ro', label='train loss')
plt.legend()
plt.show();

### LSTM (Long-Short Term Memory)


![lstm](images/lstm.png)

In [ ]:
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

from sklearn.metrics import mean_squared_error as mse

In [ ]:
seq_in=np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

n_in=len(seq_in)

seq_in=seq_in.reshape((1, n_in, 1))



seq_out=seq_in[:, 1:, :]
n_out=n_in-1

In [ ]:
# modelo
modelo=Sequential()

modelo.add(LSTM(100, activation='relu', input_shape=(n_in, 1)))
modelo.add(RepeatVector(n_out))

modelo.add(LSTM(100, activation='relu', return_sequences=True))
modelo.add(TimeDistributed(Dense(1)))

modelo.add(LSTM(100, activation='relu', return_sequences=True))
modelo.add(TimeDistributed(Dense(1)))

modelo.add(LSTM(100, activation='relu', return_sequences=True))
modelo.add(TimeDistributed(Dense(1)))

modelo.add(LSTM(100, activation='relu', return_sequences=True))
modelo.add(TimeDistributed(Dense(1)))

modelo.add(LSTM(100, activation='relu', return_sequences=True))
modelo.add(TimeDistributed(Dense(1)))

modelo.add(LSTM(100, activation='relu', return_sequences=True))
modelo.add(TimeDistributed(Dense(1)))

modelo.add(LSTM(100, activation='relu', return_sequences=True))
modelo.add(TimeDistributed(Dense(1)))

modelo.add(LSTM(100, activation='relu', return_sequences=True))
modelo.add(TimeDistributed(Dense(1)))

modelo.compile(optimizer='adam', loss='mse')

In [ ]:
modelo.fit(seq_in, seq_out, epochs=300, verbose=0)

modelo.summary()

In [ ]:
y_pred=modelo.predict(seq_in, verbose=0)

y_pred[0, :, 0]

In [ ]:
mse(seq_in[0,:,0][1:], y_pred[0, :, 0])**0.5